# Creating DataFrames for Each State

This file creates DataFrames for each state with 2020 Presidential Election Data and Total COVID-19 Cases as of mid-December.

In [1]:
# Imports - these are provided for you. Do not import any other packages.
import pandas as pd
import requests
import bs4
import time
from bs4 import BeautifulSoup

### Creating initial DataFrame with election data

Data for 2020 Presidential Election was scraped from https://abcnews.go.com/Elections/california-county-presidential-election-results-2020

In [2]:
# List of states to iterate through
state_list = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New-Hampshire','New-Jersey','New-Mexico','New-York','North-Carolina','North-Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode-Island','South-Carolina','South-Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West-Virginia','Wisconsin','Wyoming']

# Dictionary to store all DataFrames for each state
df_dictionary = {}

# Iterate through state list
for state in state_list:
    
    # Skip the special case
    if(state == 'Alaska'):
        continue
    
    # Create lists to hold county names, democrat votes and percent, republican votes and percent
    county_list, dem_per, dem_votes, rep_per, rep_votes = [], [], [], [], []
    
    # Build URL to scrape data about election
    election_page = 'https://abcnews.go.com/Elections/' + state.lower() + '-county-presidential-election-results-2020'
    page = requests.get(election_page).text
    soup = BeautifulSoup(page, 'html.parser')
    divs = soup.findAll('div', class_='CountyResults_result')
    
    # Temporary 1 second sleep timer to not overwhelm website with requests
    time.sleep(1)
    
    # Iterate through html and find data for each county
    for div in divs:
        
        # Save county name
        name = div.find('h2').text
        
        # Special case for named county in Louisiana
        if(state == 'Louisiana' and name == 'La Salle Parish'):
            name = 'LaSalle Parish';
        county_list.append(name)
    
        # Democrats info
        democrats = div.find('tr', class_='ElectionsTable__Row ResultsTable--counting ResultsTable__row--democrats')
        dems_vote = int(democrats.find('td', class_='ElectionsTable__Cell ResultsTable__votes').text.replace(',',''))
        dems_per = int(democrats.find('td', class_='ElectionsTable__Cell ResultsTable__percent').text.replace('%',''))
    
        # Append info scraped to lists
        dem_votes.append(dems_vote)
        dem_per.append(dems_per)

        # Republicans info
        republicans = div.find('tr', class_='ElectionsTable__Row ResultsTable--counting ResultsTable__row--republicans')
        reps_vote = int(republicans.find('td', class_='ElectionsTable__Cell ResultsTable__votes').text.replace(',',''))
        reps_per = int(republicans.find('td', class_='ElectionsTable__Cell ResultsTable__percent').text.replace('%',''))
    
        # Append info scraped to lists
        rep_votes.append(reps_vote)
        rep_per.append(reps_per)
    
    # Create DataFrames from lists for current state
    df = pd.DataFrame()
    df['County'] = county_list
    df['Democrat Votes'] = dem_votes
    df['Democrat %'] = dem_per
    df['Republican Votes'] = rep_votes
    df['Republican %'] = rep_per
    df.set_index('County', inplace=True)
    
    # Save to dictionary with Key as state and Value as DataFrame
    df_dictionary[state] = df
    
    print('Saving dataframe for ' + state)

Saving dataframe for Alabama
Saving dataframe for Arizona
Saving dataframe for Arkansas
Saving dataframe for California
Saving dataframe for Colorado
Saving dataframe for Connecticut
Saving dataframe for Delaware
Saving dataframe for Florida
Saving dataframe for Georgia
Saving dataframe for Hawaii
Saving dataframe for Idaho
Saving dataframe for Illinois
Saving dataframe for Indiana
Saving dataframe for Iowa
Saving dataframe for Kansas
Saving dataframe for Kentucky
Saving dataframe for Louisiana
Saving dataframe for Maine
Saving dataframe for Maryland
Saving dataframe for Massachusetts
Saving dataframe for Michigan
Saving dataframe for Minnesota
Saving dataframe for Mississippi
Saving dataframe for Missouri
Saving dataframe for Montana
Saving dataframe for Nebraska
Saving dataframe for Nevada
Saving dataframe for New-Hampshire
Saving dataframe for New-Jersey
Saving dataframe for New-Mexico
Saving dataframe for New-York
Saving dataframe for North-Carolina
Saving dataframe for North-Dakot

### Appending population data to existing DataFrames

Data for Population was scrape from 
https://worldpopulationreview.com/us-counties/states/ca


In [3]:
# List of state abbreviations to iterate through
state_abrv = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

# Iterate through state list
for x in range(50):
    
    # Skip the special case
    if(state_abrv[x] == 'AK'):
        continue
    
    # Temporary 1 second sleep timer to not overwhelm website with requests
    time.sleep(1)
    
    # Build URL to scrape data about population
    population_page = 'https://worldpopulationreview.com/us-counties/states/' + state_abrv[x].lower()
    page = requests.get(population_page).text
    soup = BeautifulSoup(page, 'html.parser')
    table_pop = soup.findAll('tr')

    # Create lists to hold county names and population
    county_list, pop = [], []
    
    # Skip heading in population table
    heading = True
    
    # Iterate through html and find info for each county
    for row in table_pop:
        
        # Skip heading in table
        if heading:
            heading = False
            continue
    
        # Find each column in the row
        row = row.findAll('td')
    
        # Save county name
        county_name = row[0].find('a').text
        
        # Special case for accented county in New Mexico
        if state_abrv[x] == 'NM' and county_name == 'DoÃ±a Ana County':
            county_name = 'Dona Ana County'
        county_list.append(county_name)
    
        # Save population info
        pops = int(row[1].find(text=True).replace(',',''))
        pop.append(pops)

    # Create DataFrame for population for each county
    pops_df = pd.DataFrame()
    pops_df['County'] = county_list
    pops_df['Population'] = pop
    pops_df.set_index('County', inplace=True)
    
    # Combine population data to existing DataFrames with voter information
    df_dictionary[state_list[x]] = pd.concat([df_dictionary[state_list[x]],pops_df], axis=1, sort=True)
    print('Adding population to ' + state_list[x])

Adding population to Alabama
Adding population to Arizona
Adding population to Arkansas
Adding population to California
Adding population to Colorado
Adding population to Connecticut
Adding population to Delaware
Adding population to Florida
Adding population to Georgia
Adding population to Hawaii
Adding population to Idaho
Adding population to Illinois
Adding population to Indiana
Adding population to Iowa
Adding population to Kansas
Adding population to Kentucky
Adding population to Louisiana
Adding population to Maine
Adding population to Maryland
Adding population to Massachusetts
Adding population to Michigan
Adding population to Minnesota
Adding population to Mississippi
Adding population to Missouri
Adding population to Montana
Adding population to Nebraska
Adding population to Nevada
Adding population to New-Hampshire
Adding population to New-Jersey
Adding population to New-Mexico
Adding population to New-York
Adding population to North-Carolina
Adding population to North-Dakot

### Appending population density data to existing DataFrames

They say most of a Data Scientist's time is spent cleaning the data... 
<br>
<sup>My sanity was tested during this portion.</sup>
<br>
Data for Population Density was scraped from http://www.usa.com/rank/california-state--population-density--county-rank.htm

In [4]:
# Iterate through state_list and state_abrv
for x in range(50):
   
    # Skip the special case
    if(state_abrv[x] == 'AK'):
        continue

    # Temporary 1 second sleep timer to not overwhelm website with requests
    time.sleep(1)
    
    # Build URL to scrape data about population density
    pop_density_url = 'http://www.usa.com/rank/' + state_list[x].lower() + '-state--population-density--county-rank.htm'
    page = requests.get(pop_density_url).text
    soup = BeautifulSoup(page, 'html.parser')
    
    # Grab the table from the website
    pop_dens_table = soup.findAll('table')
    rows = pop_dens_table[1].findAll('tr')

    # Create lists to hold county names and population
    county_list, pop_density = [], []

    # Skip heading in population table
    heading = True

    # Iterate through html and find info for each county
    for row in rows:
    
        # Skip heading in table
        if heading:
            heading = False
            continue
    
        # Find each column in the row
        content = row.findAll('td')
        
        # Save county name
        county = content[2].find('a').text.replace(', ' + state_abrv[x], ' County').replace('Saint', 'St.')
        
        # Special cases for many states...
        if state_list[x] == 'Alabama':
            if county == 'De Kalb County':
                county = 'DeKalb County'
        
        elif state_list[x] == 'Florida':
            if county == 'De Soto County':
                county = 'DeSoto County'

        elif state_list[x] == 'Georgia':
            if county == 'Dekalb County':
                county = 'DeKalb County'
            elif county == 'Mcduffie County':
                county = 'McDuffie County'
            elif county == 'Mcintosh County':
                county = 'McIntosh County'

        elif state_list[x] == 'Illinois':
            if county == 'Dekalb County':
                county = 'DeKalb County'
            elif county == 'Dewitt County':
                county = 'De Witt County'
            elif county == 'Dupage County':
                county = 'DuPage County'
            elif county == 'La Salle County':
                county = 'LaSalle County'
            elif county == 'Mcdonough County':
                county = 'McDonough County'
            elif county == 'Mchenry County':
                county = 'McHenry County'
            elif county == 'Mclean County':
                county = 'McLean County'
        
        elif state_list[x] == 'Indiana':
            if county == 'De Kalb County':
                county = 'DeKalb County'
            elif county == 'La Porte County':
                county = 'LaPorte County'
            elif county == 'Lagrange County':
                county = 'LaGrange County'
            elif county == 'St Joseph County':
                county = 'St. Joseph County'

        elif state_list[x] == 'Iowa':
            if county == 'Obrien County':
                county = 'O\'Brien County'
        
        elif state_list[x] == 'Kansas':
            if county == 'Mcpherson County':
                county = 'McPherson County'

        elif state_list[x] == 'Kentucky':
            if county == 'Mccracken County':
                county = 'McCracken County'
            elif county == 'Mccreary County':
                county = 'McCreary County'
            elif county == 'Mclean County':
                county = 'McLean County'
        
        elif state_list[x] == 'Louisiana':
            county = county.replace('County', 'Parish')
            if county == 'La Salle Parish':
                county = 'LaSalle Parish'
            elif county == 'St John The Baptist Parish':
                county = 'St. John the Baptist Parish'

        elif state_list[x] == 'Maryland':
            if county == 'Baltimore City County':
                county = 'Baltimore city'
            elif county == 'Prince Georges County':
                county = 'Prince George\'s County'
            elif county == 'Queen Annes County':
                county = 'Queen Anne\'s County'
            elif county == 'St. Marys County':
                county = 'St. Mary\'s County'
        
        elif state_list[x] == 'Minnesota':
            if county == 'Lac Qui Parle County':
                county = 'Lac qui Parle County'
            elif county == 'Lake Of The Woods County':
                county = 'Lake of the Woods County'
            elif county == 'Mcleod County':
                county = 'McLeod County'
        
        elif state_list[x] == 'Mississippi':
            if county == 'Desoto County':
                county = 'DeSoto County'
        
        elif state_list[x] == 'Missouri':
            if county == 'Dekalb County':
                county = 'DeKalb County'
            elif county == 'Mcdonald County':
                county = 'McDonald County'
            elif county == 'St. Louis City County':
                county = 'St. Louis city'
            elif county == 'St.e Genevieve County':
                county = 'Ste. Genevieve County'
        
        elif state_list[x] == 'Montana':
            if county == 'Lewis And Clark County':
                county = 'Lewis and Clark County'
            elif county == 'Mccone County':
                county = 'McCone County'
        
        elif state_list[x] == 'Nebraska':
            if county == 'Mcpherson County':
                county = 'McPherson County'
                
        elif state_list[x] == 'Nevada':
            if county == 'Carson City County':
                county = 'Carson City'
                
        elif state_list[x] == 'New-Mexico':
            if county == 'Mckinley County':
                county = 'McKinley County'
        
        elif state_list[x] == 'North-Carolina':
            if county == 'Mcdowell County':
                county = 'McDowell County'
        
        elif state_list[x] == 'North-Dakota':
            if county == 'Lamoure County':
                county = 'LaMoure County'
            elif county == 'Mchenry County':
                county = 'McHenry County'
            elif county == 'Mcintosh County':
                county = 'McIntosh County'
            elif county == 'Mckenzie County':
                county = 'McKenzie County'
            elif county == 'Mclean County':
                county = 'McLean County'
                
        elif state_list[x] == 'Oklahoma':
            if county == 'Mcclain County':
                county = 'McClain County'
            elif county == 'Mccurtain County':
                county = 'McCurtain County'
            elif county == 'Mcintosh County':
                county = 'McIntosh County'
                
        elif state_list[x] == 'Pennsylvania':
            if county == 'Mckean County':
                county = 'McKean County'
            
        elif state_list[x] == 'South-Carolina':
            if county == 'Mccormick County':
                county = 'McCormick County'
                
        elif state_list[x] == 'South-Dakota':
            if county == 'Mccook County':
                county = 'McCook County'
            elif county == 'Mcpherson County':
                county = 'McPherson County'
            elif county == 'Shannon County':
                county = 'Oglala Lakota County'
        
        elif state_list[x] == 'Tennessee':
            if county == 'Dekalb County':
                county = 'DeKalb County'
            elif county == 'Mcminn County':
                county = 'McMinn County'
            elif county == 'Mcnairy County':
                county = 'McNairy County'
                
        elif state_list[x] == 'Texas':
            if county == 'De Witt County':
                county = 'DeWitt County'
            elif county == 'Mcculloch County':
                county = 'McCulloch County'
            elif county == 'Mclennan County':
                county = 'McLennan County'
            elif county == 'Mcmullen County':
                county = 'McMullen County'

        elif state_list[x] == 'Virginia':
            county = county.replace('City County', 'city')
            if county == 'Bristol County':
                county = 'Bristol city'
            elif county == 'Charles city':
                county = 'Charles City County'
            elif county == 'Isle Of Wight County':
                county = 'Isle of Wight County'
            elif county == 'James city':
                county = 'James City County'
            elif county == 'King And Queen County':
                county = 'King and Queen County'
            elif county == 'Radford County':
                county = 'Radford city'
            elif county == 'Salem County':
                county = 'Salem city'
        
        elif state_list[x] == 'West-Virginia':
            if county == 'Mcdowell County':
                county = 'McDowell County'
                
        elif state_list[x] == 'Wisconsin':
            if county == 'Fond Du Lac County':
                county = 'Fond du Lac County'
        
        # Save density info
        density = float(content[1].text.replace(',', '').replace('/sq mi', ''))
    
        # Append saved info to lists
        county_list.append(county)
        pop_density.append(density)
    
    # Special case where density data was missing
    if state_list[x] == 'Virginia':
        county_list.append('Emporia city')
        pop_density.append(float(776.3))
    
    # Create DataFrame for population density for each county
    density_df = pd.DataFrame()
    density_df['County'] = county_list
    density_df['Population Density (per sq mi)'] = pop_density
    density_df.set_index('County', inplace=True)
    
    # Combine population density data to existing DataFrames with voter information
    df_dictionary[state_list[x]] = pd.concat([df_dictionary[state_list[x]], density_df], axis=1, sort=True)
    print('Adding population density data to '+ state_list[x])

Adding population density data to Alabama
Adding population density data to Arizona
Adding population density data to Arkansas
Adding population density data to California
Adding population density data to Colorado
Adding population density data to Connecticut
Adding population density data to Delaware
Adding population density data to Florida
Adding population density data to Georgia
Adding population density data to Hawaii
Adding population density data to Idaho
Adding population density data to Illinois
Adding population density data to Indiana
Adding population density data to Iowa
Adding population density data to Kansas
Adding population density data to Kentucky
Adding population density data to Louisiana
Adding population density data to Maine
Adding population density data to Maryland
Adding population density data to Massachusetts
Adding population density data to Michigan
Adding population density data to Minnesota
Adding population density data to Mississippi
Adding populati

### Appending COVID-19 data to existing DataFrames 

Data for COVID-19 was scraped from https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/california

In [5]:
# Iterate through list of states used earlier for election data
for state in state_list:
    
     # Temporary 1 second sleep timer to not overwhelm website with requests
    time.sleep(1)
    
    # Skip the special case
    if(state == 'Alaska'):
        continue
    
    # Build URL to scrape data about total COVID-19 cases 
    covid_page = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/' + state.lower()
    page = requests.get(covid_page).text
    soup = BeautifulSoup(page, 'html.parser')
    covid_table = soup.find('tbody').findAll('tr')
    
    # Create list to save number of total cases in each county
    total_cases = []

    # Iterate through html and find info for each county
    for row in covid_table:
        
        # Save cases info
        cases = int(row.find('td').text.replace(',',''))
        total_cases.append(cases)
    
    # Add new column 'Total COVID-19 Cases' to existing DataFrames for each state
    df_dictionary[state]['Total COVID-19 Cases'] = total_cases
    print('Adding COVID-19 data to ' + state)

Adding COVID-19 data to Alabama
Adding COVID-19 data to Arizona
Adding COVID-19 data to Arkansas
Adding COVID-19 data to California
Adding COVID-19 data to Colorado
Adding COVID-19 data to Connecticut
Adding COVID-19 data to Delaware
Adding COVID-19 data to Florida
Adding COVID-19 data to Georgia
Adding COVID-19 data to Hawaii
Adding COVID-19 data to Idaho
Adding COVID-19 data to Illinois
Adding COVID-19 data to Indiana
Adding COVID-19 data to Iowa
Adding COVID-19 data to Kansas
Adding COVID-19 data to Kentucky
Adding COVID-19 data to Louisiana
Adding COVID-19 data to Maine
Adding COVID-19 data to Maryland
Adding COVID-19 data to Massachusetts
Adding COVID-19 data to Michigan
Adding COVID-19 data to Minnesota
Adding COVID-19 data to Mississippi
Adding COVID-19 data to Missouri
Adding COVID-19 data to Montana
Adding COVID-19 data to Nebraska
Adding COVID-19 data to Nevada
Adding COVID-19 data to New-Hampshire
Adding COVID-19 data to New-Jersey
Adding COVID-19 data to New-Mexico
Adding C

# Examples of DataFrames

Sanity checks

In [6]:
df_dictionary['New-Jersey']

,Democrat Votes,Democrat %,Republican Votes,Republican %,Population,Population Density (per sq mi),Total COVID-19 Cases
Atlantic County,73808,53,64438,46,263351,409.8,9758
Bergen County,285967,58,204417,41,931588,3731.5,40879
Burlington County,154595,59,103345,39,445953,549.1,17179
Camden County,175065,66,86207,33,506589,2255.4,24003
Cape May County,23941,41,33158,57,91632,155.2,1995
Cumberland County,32742,52,28952,46,148419,232.3,6277
Essex County,266820,77,75475,22,799380,6091.3,41471
Gloucester County,86702,50,83340,48,291747,859.2,11597
Hudson County,181452,72,65698,26,672851,10509.5,37521
Hunterdon County,39457,47,43153,51,123935,289.7,3077


In [7]:
df_dictionary['California']

,Democrat Votes,Democrat %,Republican Votes,Republican %,Population,Population Density (per sq mi),Total COVID-19 Cases
Alameda County,617659,80,136309,18,1675900,1898.5,38218
Alpine County,476,64,244,33,1169,1.6,5
Amador County,8153,36,13585,61,40099,61.3,753
Butte County,50426,49,48730,48,208033,132.1,4912
Calaveras County,10046,37,16518,61,46112,43.3,642
Colusa County,3234,41,4554,57,21676,18.5,991
Contra Costa County,416386,72,152877,26,1156530,1345.2,30685
Del Norte County,4677,41,6461,56,27884,22.8,571
El Dorado County,51621,44,61838,53,194940,101.6,4085
Fresno County,193025,53,164464,45,1006250,157.8,44114


# Export DataFrames as CSV files for analysis

In [16]:
for state in state_list:
    if state == 'Alaska':
        continue
    df_dictionary[state].to_csv(state + '_csv')
